In [1]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import statsmodels.api as sm
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

# Create models folder
os.makedirs('models_ensemble', exist_ok=True)

print("All libraries ready!")

All libraries ready!


In [2]:
# Cell 2: Load and Initial Clean
df = pd.read_csv('BetelPrice.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df = df.sort_values('Date').reset_index(drop=True)

# Remove garbage columns
df = df.drop(columns=['Unnamed: 6', ' '], errors='ignore')

# Basic cleaning
df = df.dropna(subset=['Price'])
df = df[df['Price'] > 0].reset_index(drop=True)

print(f"Total rows after initial clean: {len(df)}")

Total rows after initial clean: 9429


In [3]:
# Cell 3: Outlier Removal per Commercial Type (IQR Method)
commercial_types = ['Peedunu', 'Keti', 'Kanda', 'Korikan']
df_clean = pd.DataFrame()

for ctype in commercial_types:
    print(f"\n--- {ctype} ---")
    subset = df[df['Commercial Type'] == ctype].copy()
    print(f"Original rows: {len(subset)} | Price range: {subset['Price'].min()} - {subset['Price'].max()}")

    # IQR outlier detection
    Q1 = subset['Price'].quantile(0.25)
    Q3 = subset['Price'].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    before = len(subset)
    subset = subset[(subset['Price'] >= lower) & (subset['Price'] <= upper)]
    after = len(subset)

    print(f"After outlier removal: {after} rows (removed {before - after})")
    print(f"New price range: {subset['Price'].min()} - {subset['Price'].max()}")

    df_clean = pd.concat([df_clean, subset], ignore_index=True)

df = df_clean
print(f"\nFinal total rows after outlier removal across all types: {len(df)}")


--- Peedunu ---
Original rows: 3776 | Price range: 200 - 30000
After outlier removal: 3590 rows (removed 186)
New price range: 200 - 12000

--- Keti ---
Original rows: 1888 | Price range: 50 - 10000
After outlier removal: 1812 rows (removed 76)
New price range: 50 - 5000

--- Kanda ---
Original rows: 1888 | Price range: 250 - 20000
After outlier removal: 1785 rows (removed 103)
New price range: 250 - 11000

--- Korikan ---
Original rows: 1877 | Price range: 20 - 4000
After outlier removal: 1757 rows (removed 120)
New price range: 20 - 1800

Final total rows after outlier removal across all types: 8944


In [4]:
# Cell 4: Training Loop with LSTM + SARIMA Ensemble
time_steps = 60
results = {}

print("\nStarting training with LSTM + SARIMA ensemble...\n")

for ctype in commercial_types:
    print("="*70)
    print(f"TRAINING ENSEMBLE MODEL FOR: {ctype.upper()}")
    print("="*70)

    subset = df[df['Commercial Type'] == ctype].copy()
    print(f"Rows: {len(subset)}")

    if len(subset) < 300:
        print(f"Skipping {ctype} - insufficient data after cleaning.\n")
        continue

    # Add cyclical date features
    subset['Month_sin'] = np.sin(2 * np.pi * subset['Date'].dt.month / 12)
    subset['Month_cos'] = np.cos(2 * np.pi * subset['Date'].dt.month / 12)
    subset['DayOfYear_sin'] = np.sin(2 * np.pi * subset['Date'].dt.dayofyear / 365.25)
    subset['DayOfYear_cos'] = np.cos(2 * np.pi * subset['Date'].dt.dayofyear / 365.25)

    # Encode categoricals
    cat_cols = ['District', 'Market Type', 'Quality Grade']
    subset_encoded = pd.get_dummies(subset, columns=cat_cols, dtype=float, drop_first=True)

    # Features
    feature_cols = ['Price'] + [col for col in subset_encoded.columns if col not in ['Date', 'Price', 'Commercial Type']]
    data = subset_encoded[feature_cols].values.astype(np.float32)

    # Log + scale Price
    data_log = data.copy()
    data_log[:, 0] = np.log(data[:, 0])
    scaler = MinMaxScaler()
    data_scaled = data_log.copy()
    data_scaled[:, 0] = scaler.fit_transform(data_log[:, [0]]).flatten()

    # Create sequences for LSTM
    X, y = [], []
    for i in range(len(data_scaled) - time_steps):
        X.append(data_scaled[i:i + time_steps])
        y.append(data_scaled[i + time_steps, 0])
    X = np.array(X)
    y = np.array(y)

    if len(X) < 50:
        print(f"Not enough sequences for {ctype}.\n")
        continue

    split_idx = int(0.8 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # === LSTM Model ===
    lstm_model = Sequential()
    lstm_model.add(LSTM(64, return_sequences=True, input_shape=(time_steps, X.shape[2])))
    lstm_model.add(Dropout(0.3))
    lstm_model.add(LSTM(64))
    lstm_model.add(Dropout(0.3))
    lstm_model.add(Dense(32, activation='relu'))
    lstm_model.add(Dense(1))

    lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

    early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=0)

    lstm_model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.1,
                   callbacks=[early_stop], verbose=0)

    # LSTM predictions
    lstm_pred_scaled = lstm_model.predict(X_test, verbose=0)
    lstm_pred = np.exp(scaler.inverse_transform(lstm_pred_scaled))

    # === SARIMA Model ===
    price_series = subset['Price'].values  # Use original cleaned prices
    try:
        sarima_model = sm.tsa.SARIMAX(price_series, order=(1,1,1), seasonal_order=(1,1,1,12))
        sarima_fit = sarima_model.fit(disp=False)
        sarima_pred = sarima_fit.forecast(steps=len(X_test))
    except:
        print(f"SARIMA failed for {ctype}. Using LSTM only.")
        sarima_pred = np.full_like(lstm_pred, np.mean(price_series[-100:]))  # fallback

    # === Ensemble: Simple Average ===
    ensemble_pred = 0.5 * lstm_pred.flatten() + 0.5 * sarima_pred

    y_true = np.exp(scaler.inverse_transform(y_test.reshape(-1, 1))).flatten()

    rmse = np.sqrt(mean_squared_error(y_true, ensemble_pred))
    mae = mean_absolute_error(y_true, ensemble_pred)
    mape = np.mean(np.abs((y_true - ensemble_pred) / y_true)) * 100

    print(f"ENSEMBLE RESULTS → RMSE: {rmse:.0f} | MAE: {mae:.0f} | MAPE: {mape:.1f}%\n")

    results[ctype] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

    # Save models
    lstm_model.save(f'models_ensemble/lstm_{ctype}.h5')
    joblib.dump(scaler, f'models_ensemble/scaler_{ctype}.pkl')
    joblib.dump(sarima_fit, f'models_ensemble/sarima_{ctype}.pkl')
    print(f"Models saved for {ctype}\n")


Starting training with LSTM + SARIMA ensemble...

TRAINING ENSEMBLE MODEL FOR: PEEDUNU
Rows: 3590


ENSEMBLE RESULTS → RMSE: 2574 | MAE: 2065 | MAPE: 36.7%

Models saved for Peedunu

TRAINING ENSEMBLE MODEL FOR: KETI
Rows: 1812


ENSEMBLE RESULTS → RMSE: 1118 | MAE: 911 | MAPE: 43.4%

Models saved for Keti

TRAINING ENSEMBLE MODEL FOR: KANDA
Rows: 1785


ENSEMBLE RESULTS → RMSE: 1894 | MAE: 1513 | MAPE: 28.2%

Models saved for Kanda

TRAINING ENSEMBLE MODEL FOR: KORIKAN
Rows: 1757


ENSEMBLE RESULTS → RMSE: 338 | MAE: 281 | MAPE: 79.3%

Models saved for Korikan



In [5]:
# Cell 5: Final Summary
print("\n" + "="*80)
print("FINAL ENSEMBLE PERFORMANCE SUMMARY (After Outlier Removal)")
print("="*80)
for ctype in commercial_types:
    if ctype in results:
        m = results[ctype]
        print(f"{ctype:9} → MAPE: {m['MAPE']:5.1f}%  |  MAE: {m['MAE']:7.0f}  |  RMSE: {m['RMSE']:7.0f}")
    else:
        print(f"{ctype:9} → Skipped")
print("="*80)
print("Improvement expected over previous version!")


FINAL ENSEMBLE PERFORMANCE SUMMARY (After Outlier Removal)
Peedunu   → MAPE:  36.7%  |  MAE:    2065  |  RMSE:    2574
Keti      → MAPE:  43.4%  |  MAE:     911  |  RMSE:    1118
Kanda     → MAPE:  28.2%  |  MAE:    1513  |  RMSE:    1894
Korikan   → MAPE:  79.3%  |  MAE:     281  |  RMSE:     338
Improvement expected over previous version!
